In [1]:
import torch
from torch.nn import functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import OneCycleLR  #
# from torch.utils.data import TensorDataset, \
#     DataLoader  # Own stuff set of data quality checks, tensor shapes might be different dataloader loads the tensor,
import pytorch_lightning as pl  #
from pytorch_lightning.callbacks.early_stopping import \
    EarlyStopping  # early stop when you reach optimum loss, 3 times in a row gradient descent
from pytorch_lightning.callbacks import LearningRateMonitor  # delta (loss / accuracy)
from pytorch_lightning.loggers import MLFlowLogger  # Model tracking
import torch
import torch.nn as nn
from transformers import DistilBertModel
from data_preprocessing import injestDataset, downSample
from data_cleaning import feature_cleaner, cleaningPreprocess
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

import torchmetrics
import pytorch_lightning as pl
from transformers import BertTokenizer
# from transformers import DistilBertModel

In [2]:
data = injestDataset()

In [3]:
df = downSample(data,"Recommended IND",1000)

In [4]:
df

,trueIndex,Review Text,Rating,Recommended IND
0,12849,I've been looking for sometime for a spring/su...,5,1
1,4578,Great spring time dress that looks good with a...,5,1
2,6169,This material of this shirt is soft and appeal...,4,1
3,2942,This dress is so light and twirly and the fabr...,4,1
4,2769,I bought this shirt for my 16 year old daughte...,4,1
...,...,...,...,...
995,12936,"This is a beautiful dress. however, it looked ...",1,0
996,258,"Fits well through the shoulders and arms, but ...",3,0
997,19148,I love maeve dresses but there are several iss...,3,0
998,20477,"Okay, so this top has its redeeming qualities,...",3,0


In [5]:
df = cleaningPreprocess(df, "Review Text")

In [6]:
df = df.rename(columns={"Recommended IND": "targetOne", "Review Text": "feature", "Rating": "targetTwo"})

In [7]:
df

,trueIndex,feature,targetTwo,targetOne
0,12849,ive looking sometime springsummer jacket wear ...,5,1
1,4578,great spring time dress looks good thin top ca...,5,1
2,6169,material shirt soft appealing great fall side ...,4,1
3,2942,dress light twirly fabric airy pretty sadly fi...,4,1
4,2769,bought shirt 16 year old daughter looked adora...,4,1
...,...,...,...,...
995,12936,beautiful dress however looked awful ordered m...,1,0
996,258,fits well shoulders arms zero waist looks like...,3,0
997,19148,love maeve dresses several issues one taste 1 ...,3,0
998,20477,okay top redeeming qualities promise gave 3 st...,3,0


In [8]:
# features = df.loc[:,df.columns.isin(["feature"])]
# targetOne = df.loc[:,df.columns.isin(["targetOne"])]
# targetTwo = df.loc[:,df.columns.isin(["targetTwo"])]

In [9]:
# from transformers import BertTokenizer

# tokenizer = 


In [10]:
# max_length = 12  # Max sequence length after tokenization
# padding_side = "max_length"  # ['longest', 'max_length', 'do_not_pad']

# input_ids = tokenizer("New man in the house", max_length=max_length, padding=padding_side, truncation=True)


In [11]:
# input_ids

In [12]:
# df

In [13]:
# tokenized_texts = []
# for text in df['text']:
#     encoded_inputs = tokenizer(
#             text,
#             add_special_tokens=True,
#             max_length=max_length,
#             padding='max_length',
#             truncation=True,
#             # return_tensors='pt'
#         )
#         tokenized_texts.append(encoded_inputs)

In [14]:
# attention_mask = [1] * len(input_ids)  # 1 for actual tokens, 0 for padding tokens

In [15]:
df

,trueIndex,feature,targetTwo,targetOne
0,12849,ive looking sometime springsummer jacket wear ...,5,1
1,4578,great spring time dress looks good thin top ca...,5,1
2,6169,material shirt soft appealing great fall side ...,4,1
3,2942,dress light twirly fabric airy pretty sadly fi...,4,1
4,2769,bought shirt 16 year old daughter looked adora...,4,1
...,...,...,...,...
995,12936,beautiful dress however looked awful ordered m...,1,0
996,258,fits well shoulders arms zero waist looks like...,3,0
997,19148,love maeve dresses several issues one taste 1 ...,3,0
998,20477,okay top redeeming qualities promise gave 3 st...,3,0


In [16]:
# Here is your dataset
class MyDataset(Dataset):
    def __init__(self, df,transform=None):
        self.input_data = df
        self.max_length = 30
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.tokenize_dataframe()
        # self.split_columns()
        # # Perform one-hot encoding on the features
        # self.column_transformer = None
        # self.new_features = None
        # self.label_binarizer = LabelEncoder()
        # self.new_targetOne =  None
        # self.new_targetTwo =  Non

        # self.features = self.raw_features.loc[self.indices]
        # self.targetOne = self.raw_targetOne.loc[self.indices]   
        # self.targetTwo = self.raw_targetTwo.loc[self.indices]  

        # self.transform = transform 

    def tokenize_dataframe(self):
        tokenized_texts = []

        for text in self.input_data['feature']:
            encoded_inputs = self.tokenizer(
                text,
                add_special_tokens=True,
                max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            tokenized_texts.append(encoded_inputs)
        
        self.input_data['tokenizedFeature'] = tokenized_texts

    def split_columns(sef):
        self.features = self.input_data["feature"]
        self.targetOne = self.input_data["targetOne"]
        self.targetTwo = self.input_data["targetTwo"]

            
    def train_val_test_idx(self):
        idx = list(range(len(self.input_data)))
        train_idx, val_test_idx = train_test_split(
                                idx,
                                train_size=0.8,
                                stratify=self.input_data["targetOne"],
                                random_state=500)
        
        test_idx,  val_idx, = train_test_split(
                                val_test_idx,
                                train_size=0.5,
                                random_state=500)
        self.train_idx = train_idx 
        self.test_idx = test_idx
        self.val_idx = val_idx,
    
        



        # return train_idx, test_idx,  val_idx 

        # if self.train:
        #     self.indices = train_idx
            
        # elif self.test:
        #     self.indices =  test_idx
            
        # elif self.val:
        #     self.indices =  val_idx
        # else :
        #     self.indices = idx
        
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, index):
        sample = self.features[index], self.targetOne[index], self.targetTwo[index] 
        # if self.transform:
        #     sample = self.transform(sample)
        return sample

    
class to_tensor:
    def __call__(self,sample):
        features, targets= sample
        return torch.tensor(features,dtype=torch.float32,device='mps:0'), torch.tensor(targets ,dtype=torch.int32,  device='mps:0')
    
    
def transformInput(trainDataset,this_dict):
    return torch.tensor(trainDataset.column_transformer.transform(this_dict),dtype=torch.float32,device='mps:0')
    
#The loader outputs a tensor float64 mac uses mps:0 and float32


In [17]:
trainDataset = MyDataset(df)

In [18]:
trainDataset.input_data

,trueIndex,feature,targetTwo,targetOne,tokenizedFeature
0,12849,ive looking sometime springsummer jacket wear ...,5,1,"[input_ids, token_type_ids, attention_mask]"
1,4578,great spring time dress looks good thin top ca...,5,1,"[input_ids, token_type_ids, attention_mask]"
2,6169,material shirt soft appealing great fall side ...,4,1,"[input_ids, token_type_ids, attention_mask]"
3,2942,dress light twirly fabric airy pretty sadly fi...,4,1,"[input_ids, token_type_ids, attention_mask]"
4,2769,bought shirt 16 year old daughter looked adora...,4,1,"[input_ids, token_type_ids, attention_mask]"
...,...,...,...,...,...
995,12936,beautiful dress however looked awful ordered m...,1,0,"[input_ids, token_type_ids, attention_mask]"
996,258,fits well shoulders arms zero waist looks like...,3,0,"[input_ids, token_type_ids, attention_mask]"
997,19148,love maeve dresses several issues one taste 1 ...,3,0,"[input_ids, token_type_ids, attention_mask]"
998,20477,okay top redeeming qualities promise gave 3 st...,3,0,"[input_ids, token_type_ids, attention_mask]"


In [19]:
trainDataset.input_data["tokenizedFeature"][1]

{'input_ids': tensor([[  101,  2307,  3500,  2051,  4377,  3504,  2204,  4857,  2327,  4003,
         10762,  4906,  2978,  6065, 21698,  5156,  2946,  2215, 11754,  5017,
         21776, 13950,  2100,  2052,  2946,  2293,  8313,  2507,  2750,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])}

In [20]:
# class TextDataset(torch.utils.data.Dataset):
#     def __init__(self, dataset, tokenizer, max_len):
#         self.tokenizer = tokenizer
#         self.max_len = max_len
#         self.encodings = None
#         self.raw_text = dataset['feature']
#         self.create_encoding()
#         self.labels = None
#         self.raw_labels = dataset['target']
#         self.create_target()
#         self.input_ids = self.encodings['input_ids']
#         self.attention_mask = self.encodings['attention_mask']
#         self.output_len = len(self.labels[0])

#     def __getitem__(self, idx):
#         item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
#         item['labels'] = self.labels[idx]
#         return item

#     def __len__(self):
#         return len(self.labels)

#     def create_encoding(self):
#         x = self.tokenizer(text=self.raw_text.tolist(),
#                            add_special_tokens=True,
#                            max_length=self.max_len,
#                            padding='max_length',
#                            return_tensors='pt',
#                            return_token_type_ids=False,
#                            return_attention_mask=True,
#                            verbose=True)
#         self.encodings = x

#     def create_target(self):
#         y = self.raw_labels
#         y = torch.tensor(np.stack(y.values))
#         self.labels = y


In [21]:
# class TextDataset(torch.utils.data.Dataset):
#     def __init__(self, dataset, tokenizer, max_len):
#         self.tokenizer = tokenizer
#         self.max_len = max_len
#         self.encodings = None
#         self.raw_text = dataset['feature']
#         self.create_encoding()
#         self.labels = None
#         self.raw_labels = dataset['target']
#         self.create_target()
#         self.input_ids = self.encodings['input_ids']
#         self.attention_mask = self.encodings['attention_mask']
#         self.output_len = len(self.labels[0])

#     def __getitem__(self, idx):
#         item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
#         item['labels'] = self.labels[idx]
#         return item

#     def __len__(self):
#         return len(self.labels)

#     def create_encoding(self):
#         x = self.tokenizer(text=self.raw_text.tolist(),
#                            add_special_tokens=True,
#                            max_length=self.max_len,
#                            padding='max_length',
#                            return_tensors='pt',
#                            return_token_type_ids=False,
#                            return_attention_mask=True,
#                            verbose=True)
#         self.encodings = x

#     def create_target(self):
#         y = self.raw_labels
#         y = torch.tensor(np.stack(y.values))
#         self.labels = y


In [22]:

# # Here is your PyTorch Lightning model
# class LitNetwork(pl.LightningModule):
#     def __init__(self, trainDataLoader, testDataLoader, valDataLoader):
#         super(LitNetwork, self).__init__()
#         self.train_ds = trainDataLoader
#         self.test_ds = testDataLoader
#         self.val_ds = valDataLoader
#         self.linear1 = nn.Linear(48, 25)
#         self.linear2 = nn.Linear(25, 2)
#         # self.label_binarizer = trainDataLoader.

#     def forward(self, x):
#         x = torch.sigmoid(self.linear1(x))
#         x = self.linear2(x)
#         return x

#     def training_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = nn.functional.cross_entropy(y_hat, y)
        
#         # Calculate accuracy
#         preds = torch.argmax(y_hat, dim=1)
#         acc = torchmetrics.functional.accuracy(preds,  y)

#         # Logging to TensorBoard by default
#         self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
#         self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
#         return loss
    

#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=0.02)

#     def train_dataloader(self):
#         return self.train_ds

#     def test_dataloader(self):
#         return self.test_ds  # return your dataloader
        
#     def val_dataloader(self):
#         return self.val_ds  # return your dataloader

In [23]:
# def loadModel():
#         trainDataset = MyDataset(features, targetOne, targetTwo, train=True)
#         testDataset = MyDataset(features, targetOne, targetTwo, test=True)
#         valDataset = MyDataset(features, targetOne, targetTwo, val=True)

#         trainDataLoader = DataLoader(trainDataset, batch_size=50, shuffle=True)
#         testDataLoader = DataLoader(testDataset, batch_size=50, shuffle=True)
#         valDataLoader = DataLoader(valDataset, batch_size=50, shuffle=True)

#         # model = LitNetwork(trainDataLoader,testDataLoader,valDataLoader)
#         return trainDataLoader, testDataLoader, valDataLoader

In [24]:
# x, y, z = loadModel()

In [25]:
# import numpy as np
# class MultiClassClassifier(pl.LightningModule):
#     def __init__(self, seed, output_dim, input_dim=4, hidden_dim=5, learning_rate=1e-3, max_learning_rate=0.1, total_steps=100):
#         # lower than epoch higher than iterations
#         super().__init__()
#         self.save_hyperparameters()  # save_hyperparameters
#         pl.seed_everything(seed)
#         self.model = DistilBertModel.from_pretrained('distilbert-base-uncased')
#         self.pre_classifier = torch.nn.Linear(768, 768)
#         self.dropout = torch.nn.Dropout(0.3)
#         self.classifier = torch.nn.Linear(768,output_dim)

#     def forward(self, batch):
#         # input_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
#         # input_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")
#         embeddings = self.model(batch['input_ids'], attention_mask=batch['attention_mask'],return_dict=False)[0]
#         pooler = self.pre_classifier(embeddings)
#         pooler = torch.nn.ReLU()(pooler)
#         pooler = self.dropout(pooler)
#         output = self.classifier(pooler)
#         return output

#     def configure_optimizers(self):
#         optimizer = Adam(self.parameters(), lr=self.hparams.learning_rate)
#         scheduler = OneCycleLR(optimizer, self.hparams.max_learning_rate, self.hparams.total_steps)
#         return [optimizer], [scheduler]

#     def training_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.nll_loss(logits, batch['labels'])
#         self.log("train_loss", loss, on_step=False, on_epoch=True)
#         return loss

#     def validation_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.nll_loss(logits, batch['labels'])
#         self.log("val_loss", loss)

#     def test_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.nll_loss(logits, batch['labels'])
#         self.log("test_loss", loss)

#     def predict(self, x):
#         # TODO this will be changed
#         self.eval()
#         logits = self.forward(x)
#         self.train()
#         return torch.argmax(logits, dim=1).detach().numpy()

#     # def df_to_tensor(self, df, target_col=None, format=np.float32):
#     #     if target_col is not None:
#     #         feature_cols = [col for col in df.columns if col != target_col]
#     #         tensor = TensorDataset(
#     #             torch.tensor(df[feature_cols].values.astype(format)),
#     #             torch.tensor(df[target_col].values),
#     #         )
#     #     else:
#     #         tensor = torch.tensor(df.values.astype(format))
#     #     return tensor

#     def tensor_to_loader(self, tensor, batch_size, num_workers, shuffle=True):
#         loader = DataLoader(dataset=tensor, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle)
#         return loader

#     def setup_trainer(self, experiment, run_id, max_epochs):
#         mlf_logger = MLFlowLogger(experiment_name=experiment)
#         mlf_logger._run_id = run_id
#         trainer = pl.Trainer(
#             max_epochs=max_epochs,
#             callbacks=[EarlyStopping(monitor="val_loss"), LearningRateMonitor(logging_interval="epoch")],
#             logger=mlf_logger,
#         )

#         return trainer

In [26]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.encodings = None
        self.raw_text = dataset['feature']
        self.create_encoding()
        self.labels = None
        self.raw_labels = dataset['target']
        self.create_target()
        self.input_ids = self.encodings['input_ids']
        self.attention_mask = self.encodings['attention_mask']
        self.output_len = len(self.labels[0])

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

    def create_encoding(self):
        x = self.tokenizer(text=self.raw_text.tolist(),
                           add_special_tokens=True,
                           max_length=self.max_len,
                           padding='max_length',
                           return_tensors='pt',
                           return_token_type_ids=False,
                           return_attention_mask=True,
                           verbose=True)
        self.encodings = x

    def create_target(self):
        y = self.raw_labels
        y = torch.tensor(np.stack(y.values))
        self.labels = y